In [1]:
#hide
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#hide
%load_ext autoreload
%autoreload 2

%matplotlib inline
%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [3]:
__all__ ='noop Flatten conv act_fn conv_layer kaiming_init_cnn AdaptiveConcatPool2d ParameterModule children_and_parameters children flatten_model'.split(" ")

In [ ]:
#hide
# !git clone https://github.com/prajwal-suresh13/dl_lib.git

In [ ]:
from dl_lib.core.utils import *

In [ ]:
def noop(x): return x

class Flatten(nn.Module):
  def forward(self,x): return x.view(x.size(0), -1)

In [ ]:
def conv(ni, nf, ks=3, stride=1, bias = False):
  return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

In [ ]:
act_fn = nn.ReLU(inplace=True)

In [ ]:
def conv_layer(ni, nf, ks=3, stride=1, zero_bn = False, act = True):
  bn = nn.BatchNorm2d(nf)
  nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
  layers = [conv(ni, nf, ks=ks, stride=stride), bn]
  if act:layers.append(act_fn)
  return nn.Sequential(*layers)


In [ ]:
def kaiming_init_cnn(m, uniform=False):
  if getattr(m,'bias', None) is not None: nn.init.constant_(m.bias, 0)
  if isinstance(m, (nn.Linear, nn.Conv2d)): nn.init.kaiming_normal_(m.weight) if uniform==False else nn.init.kaiming_uniform_(m.weight)
  for l in m.children():kaiming_init_cnn(l, uniform=uniform)

In [ ]:
class AdaptiveConcatPool2d(nn.Module):
  def __init__(self, sz=1):
    super().__init__()
    self.output_size = sz
    self.ap = nn.AdaptiveAvgPool2d(sz)
    self.mp = nn.AdaptiveMaxPool2d(sz)

  def forward(self, x): return torch.cat([self.mp(x), self.ap(x)],1)

In [ ]:
class ParameterModule(nn.Module):
    def __init__(self, p:nn.Parameter): super().__init__();self.val=p
    def forward(self,x):return x

In [ ]:
def children_and_parameters(m):
    children = list(m.children())
    children_p = sum([[id(p) for p in c.parameters()] for c in m.children()],[])
    for p in m.parameters():
        if id(p) not in children_p: children.append(ParameterModule(p))
    return children

In [ ]:
def children(m):return list(m.children())

In [ ]:
flatten_model = lambda m:sum(map(flatten_model,children_and_parameters(m)),[]) if len(children(m)) else [m]

In [ ]:
#hide
!pip install fire
!python dl_lib/notebook2script.py notebooks/core/models/basicmodels.ipynb dl_lib/core/models